In [1]:
# common libraries
from dotenv import load_dotenv
import os
from os import environ
import openai
from icecream import ic

# load our environment file
load_dotenv()

# define our API Key
openai.api_key = os.getenv("openai_api_key")

# Lab 0 - Support Functions

A valid message for OpenAI has two features
1. It is valid JSON.
1. It has a "role".
1. It has a "message".

We can check this with `is_valid_message`

In [2]:
from typing import List, Dict, Any

def is_valid_message(message: Dict[str, Any]) -> bool:
    """
    Check if a single message dictionary has the correct format to be sent to OpenAI.

    Args:
        message (Dict[str, Any]): A message dictionary with 'role' (str) and 'content' (str) keys.

    Returns:
        bool: True if the message is in the correct format, False otherwise.
    """
    # Check if the message dictionary has 'role' and 'content' keys of the correct types.
    if isinstance(message, dict) and 'role' in message and 'content' in message:
        if isinstance(message['role'], str) and isinstance(message['content'], str):
            return True
    return False


It is very common to have to check multiple messages, not just one.

We can check them all with `are_valid_messages`

In [3]:
def are_valid_messages(messages: List[Dict[str, Any]]) -> bool:
    """
    Check if a list of messages is in the correct format to be sent to OpenAI.

    Args:
        messages (List[Dict[str, Any]]): A list of message dictionaries.

    Returns:
        bool: True if all messages are in the correct format, False otherwise.
    """
    return all(is_valid_message(message) for message in messages)


Most chat interactions have a few parts:
1.  A list of messages to be processed together.
1. An ID for the openAI model to be used.
1. How creattive you want the generation to be.
1. The maximum number of tokens you want to use.

We encapsulate this in `simple_chat`, and get back a JSON object with a number of different values.

In [4]:
def simple_chat(messages: List[Dict[str, Any]], model: str = 'gpt-3.5-turbo', temperature: float = 0.9, max_tokens: int = 1024) -> str:
    """
    Conduct a simple chat conversation using OpenAI's GPT-3 model.

    Args:
        messages (List[Dict[str, Any]]): A list of message dictionaries, where each dictionary contains a 'role' (str)
            and 'content' (str) key-value pair representing the role of the message sender (e.g., 'system', 'user', 'assistant')
            and the content of the message.
        model (str, optional): The OpenAI model to use (default is 'gpt-3.5-turbo').
        temperature (float, optional): Controls the randomness of the response. Higher values (e.g., 0.9) make the output more random,
            while lower values (e.g., 0.2) make it more deterministic. Default is 0.9.
        max_tokens (int, optional): The maximum length of the response, measured in tokens. Default is 1024 tokens.

    Returns:
        str: The response generated by the GPT-3 model.

    Raises:
        ValueError: If the input messages are not in the correct format.

    Example:
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': 'What's the weather like today?'},
        ]
        response = simple_chat(messages)
        print(response)  # Print the generated response.
    """

    if not messages:
        raise ValueError("Input messages list cannot be empty.")

    # Check if all messages are in the correct format.
    if not are_valid_messages(messages):
        raise ValueError("Input messages must be in the format [{'role': str, 'content': str}, ...]")

    # Send the messages to OpenAI and get the response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return response


It can be very useful to look at the actual details of the response.

`show_response_detail` is a convienience function that outputs the details to the screen.

In [22]:
def show_response_detail(response: openai.openai_object.OpenAIObject):
    """
    Extracts and displays details of the first message choice from an OpenAI response object.

    This function is designed to work with response objects returned by OpenAI's language models,
    specifically with choices that contain messages with 'role' and 'content' attributes.

    Args:
        response (openai.openai_object.OpenAIObject): The OpenAI response object containing message choices.

    Returns:
        None

    Example:
        response = openai.Completion.create(
            model="gpt-3.5-turbo",
            prompt="Translate the following English text to French: 'Hello, world.'"
        )
        response_detail(response)
    """
    
    ic({response.choices[0].message.role})
    ic({response.choices[0].message.content})
    ic({response.usage.prompt_tokens})
    ic({response.usage.completion_tokens})
    ic({response.usage.total_tokens})

# Lab 0 - Example 1

## Summarizing a Message

A good job for Generative AI is generating text.  Here is a code example of exactly how you can do this with OpenAI.

** Notes **

1. We are using two different roles to definte the messages.
1. The response object contains a lot of interesting information in it.

In [28]:
long_message = """Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter. When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

# build our messages to send to openAI.  These should be well formed JSON with a ROLE and CONTENT
# system_message = {"role":"system", "content":"Summarize content you are provided in 1 sentence."}
system_message = {"role":"system", "content":"Summarize content you are provided."}
user_message = {"role":"user", "content":long_message}

summary_response = simple_chat(messages=[system_message, user_message])
summary_response

<class 'openai.openai_object.OpenAIObject'>


<OpenAIObject chat.completion id=chatcmpl-8DIJ1BLXDcg6q7PSzY5e37DtLw07x at 0x107be6390> JSON: {
  "id": "chatcmpl-8DIJ1BLXDcg6q7PSzY5e37DtLw07x",
  "object": "chat.completion",
  "created": 1698179323,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Jupiter is the largest planet in our Solar System and the fifth planet from the Sun. It is a gas giant with a mass that is one-thousandth that of the Sun, but it is two-and-a-half times more massive than all the other planets combined. Jupiter has been known to ancient civilizations since ancient times and is named after the Roman god Jupiter. It is one of the brightest objects in the night sky and can be seen with the naked eye. Its brightness is sometimes strong enough to cast visible shadows, and it is typically the third-brightest natural object in the night sky, only after the Moon and Venus."
      },
      "finish_reason": "stop"
    }
  ],
  

In [29]:
show_response_detail(summary_response)

ic| {response.choices[0].message.role}: {'assistant'}
ic| {response.choices[0].message.content}: {'Jupiter is the largest planet in our Solar System and the fifth planet from '
                                            'the Sun. It is a gas giant with a mass that is one-thousandth that of the '
                                            'Sun, but it is two-and-a-half times more massive than all the other planets '
                                            'combined. Jupiter has been known to ancient civilizations since ancient '
                                            'times and is named after the Roman god Jupiter. It is one of the brightest '
                                            'objects in the night sky and can be seen with the naked eye. Its brightness '
                                            'is sometimes strong enough to cast visible shadows, and it is typically the '
                                            'third-brightest natural object in the night sky

In [30]:
long_message = "Write a python function that will print out the most useful parts of a response object from OpenAI"
user_message = {"role":"user", "content":long_message}

code_response = simple_chat(messages=[user_message])
code_response

<class 'openai.openai_object.OpenAIObject'>


<OpenAIObject chat.completion id=chatcmpl-8DILPFUelHN5XlQryE8OQYkjKzDxB at 0x1075649a0> JSON: {
  "id": "chatcmpl-8DILPFUelHN5XlQryE8OQYkjKzDxB",
  "object": "chat.completion",
  "created": 1698179471,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Here's a Python function that will print out the most useful parts of a response object from OpenAI's GPT-3 API:\n\n```python\ndef print_useful_info(response):\n    print(\"Response Status Code:\", response.status_code)\n    print(\"Response Headers:\", response.headers)\n    print(\"Response JSON:\", response.json())\n```\n\nThis function takes a response object as an argument and prints out the following useful information:\n\n1. Response Status Code: The HTTP status code returned by the API. It indicates whether the request was successful or if there was an error.\n\n2. Response Headers: The headers returned by the API. Headers contain additional 

In [32]:
show_response_detail(code_response)

ic| {response.choices[0].message.role}: {'assistant'}
ic| {response.choices[0].message.content}: {"Here's a Python function that will print out the most useful parts of a "
                                            "response object from OpenAI's GPT-3 API:
                                           "
                                            '
                                           '
                                            '```python
                                           '
                                            'def print_useful_info(response):
                                           '
                                            '    print("Response Status Code:", response.status_code)
                                           '
                                            '    print("Response Headers:", response.headers)
                                           '
                                            '    print("Response JSON:", response.json())
      